<a href="https://colab.research.google.com/github/LyaSolis/PubMed_LAG/blob/main/1_get_articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls -a /content/drive/MyDrive/

'Colab Notebooks'   datasets   fashion	 GitHub   plant_stuff


In [ ]:
import pandas as pd
original_article_list = pd.read_csv('/content/drive/MyDrive/GitHub/PubMed_pipeline/LAG/lag_articles_list.csv')
original_article_list.head(1)

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,34674862,Right Ventricular-Pulmonary Arterial Coupling ...,"Brener MI, Grayburn P, Lindenfeld J, Burkhoff ...",JACC Cardiovasc Interv. 2021 Oct 25;14(20):223...,Brener MI,JACC Cardiovasc Interv,2021,2021/10/22,NaN,NaN,10.1016/j.jcin.2021.07.047


In [ ]:
len(original_article_list)

1173

In [ ]:
original_article_list = original_article_list.drop_duplicates()
len(original_article_list)

1173

In [ ]:
orig_pmid_pcmid_list = original_article_list[["PMID", "PMCID"]]
orig_pmid_pcmid_list[orig_pmid_pcmid_list["PMID"].isna()]

,PMID,PMCID


In [ ]:
orig_pmid_pcmid_list[orig_pmid_pcmid_list["PMCID"].isna()]

,PMID,PMCID
0,34674862,NaN
3,34600602,NaN
4,34593120,NaN
10,34167677,NaN
11,33957299,NaN
...,...,...
1168,349928,NaN
1169,1269432,NaN
1170,768906,NaN
1171,812443,NaN


In [ ]:
orig_pmid_pcmid_list[orig_pmid_pcmid_list["PMCID"].notna()]

,PMID,PMCID
1,34627582,PMC8514406
2,34624174,PMC8500343
5,34497295,PMC8426395
6,34373566,PMC8352969
7,34302745,PMC8294807
...,...,...
1067,1467051,PMC1025687
1128,3326634,PMC1386207
1147,6362688,PMC481475
1157,7020727,PMC482584


### Missing PMCIDs
928 Articles don't have PMCID included - this means using PMID we can only retrieve abstracts.  

Let's try mapping PMCIDs via pubtator file.




In [ ]:
ptt_pmc_to_pm_map = pd.read_csv('/content/drive/MyDrive/datasets/pubtator-pmids-to-pmcids.tsv', sep='\t')
ptt_pmc_to_pm_map.head(1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Journal Title,ISSN,eISSN,Year,Volume,Issue,Page,DOI,PMCID,PMID,Manuscript Id,Release Date
0,EMBO Rep,1469-221X,1469-3178,2005,6,1,1,10.1038/sj.embor.7400326,PMC1299236,15643439,NaN,live


In [ ]:
orig_merged_ppt_map = pd.merge(orig_pmid_pcmid_list, ptt_pmc_to_pm_map, on='PMID', how='left')
orig_merged_ppt_map['PMCID_x'].fillna(orig_merged_ppt_map['PMCID_y'], inplace=True)
orig_merged_ppt_map[orig_merged_ppt_map["PMCID_x"].isna()]
#No new PMIDs retrieved

,PMID,PMCID_x,Journal Title,ISSN,eISSN,Year,Volume,Issue,Page,DOI,PMCID_y,Manuscript Id,Release Date
0,34674862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34600602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34593120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,34167677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,33957299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,349928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1169,1269432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1170,768906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1171,812443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
orig_merged_ppt_map["PMCID"] = orig_merged_ppt_map["PMCID_x"]
id_list = orig_merged_ppt_map[["PMID", "PMCID"]]
len(id_list)

1173

1. Retrieve PMCIDs using this online tool to convert PMID to PMCID (200 at a time)
[https://www.ncbi.nlm.nih.gov/pmc/tools/idconv/]. 

  Results map file: [/content/drive/MyDrive/GitHub/PubMed_pipeline/LAG/LAG_missing_articles]

2. Fill in missing PMCIDs.  
  We only got 1 new PMCID

In [ ]:
id_conv_tool_results = pd.read_csv('/content/drive/MyDrive/GitHub/PubMed_pipeline/LAG/mapped_pmid_to_pmcid.txt', sep='\t')
id_conv_tool_results = id_conv_tool_results[["PMID", "PMCID"]]
conv_tool_map = pd.merge(id_list, id_conv_tool_results, on='PMID', how='left')
conv_tool_map['PMCID'] = conv_tool_map['PMCID_x'].fillna(conv_tool_map['PMCID_y'])
len(conv_tool_map)


1176

In [ ]:
conv_tool_map = conv_tool_map.drop_duplicates()
len(conv_tool_map)

1173

In [ ]:
conv_tool_map[conv_tool_map["PMCID"].isna()]

,PMID,PMCID_x,PMCID_y,PMCID
0,34674862,NaN,NaN,NaN
4,34600602,NaN,NaN,NaN
5,34593120,NaN,NaN,NaN
12,33957299,NaN,NaN,NaN
13,33952848,NaN,NaN,NaN
...,...,...,...,...
1171,349928,NaN,NaN,NaN
1172,1269432,NaN,NaN,NaN
1173,768906,NaN,NaN,NaN
1174,812443,NaN,NaN,NaN


In [ ]:
conv_tool_map = conv_tool_map[["PMID", "PMCID"]]
pcmid_list = conv_tool_map[conv_tool_map["PMCID"].notna()]
len(pcmid_list)

246

In [ ]:
pcmid_list = pcmid_list.drop_duplicates()
len(pcmid_list)

246

In [ ]:
pmid_list = conv_tool_map[conv_tool_map["PMCID"].isna()]
len(pmid_list)

927

In [ ]:
pmcid = []
pmid = []

for val in pcmid_list["PMCID"]:
  pmcid.append(val)

for val in pmid_list["PMID"]:
  pmid.append(val)

In [ ]:
print("Number Of PMCIDs Is: ", len(pmcid), "Number Of PMIDs Is: ", len(pmid))

Number Of PMCIDs Is:  246 Number Of PMIDs Is:  927


In [ ]:
with open("/content/drive/MyDrive/GitHub/PubMed_pipeline/LAG/246_pmcid.txt", "w") as outfile:
            for line in pmcid:
                outfile.write("%s\n" % line)

with open("/content/drive/MyDrive/GitHub/PubMed_pipeline/LAG/927_pmid.txt", "w") as outfile:
            for line in pmid:
                outfile.write("%s\n" % line)

USE PMCID FILE TO GET FULL TEXT ANNOTATED FILES

USE THE PMID FILE TO QUERY ABSTRACTS

## Export Annotations
Export annotated publications in batches of up to 100 in BioC, pubtator or JSON formats.

In order not to overload the PubTator server, no more than three requests per second is allowed.

Programmatically retrieve text-mined results in PubTator using web queries as follows (i.e. paste in browser):


https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=30580340,31648845,29298124,29880427,30343508,32452118,33211166,29217526,32138501,28423234,32726793,30686644,31297841,32457004,29070502,29437943,32008947,34674862,31202506,33773120,31335755,30522722,28378883,32398473,32061506,33957299,32001075,29070699,33765679,31391223,28242288,28329240,30253971,31785056,31166081,29580746,32472357,29447737,28279499,31862249,29705220,31096099,30773322,32028288,29945811,30262155,28994364,30244181,33952848,29891241,28610669,28412029,28379881,33545364,29579597,28420826,28427554,33232470,28189191,28490529,34600602,30031217,31250045,29501590,30209017,32918899,31060740,28100891,33052717,30732732,28119496,33789009,34593120,29414450,32164906,30334069,31178422,28400191,29080064,29037314,29534403,33674243,30139595,29096938,29563354,33545163,31432556,28893866,30391194,29174928,33167032,28285391,28583617,31512160,31371115,28054902,31073087,29895486,30098861,28190787




https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=32037273,28639216,31079024,32736963,31047956,28919201,33735670,30477763,28797752,26567484,24881311,24991933,27671974,23953817,27001191,24603117,24909805,29931843,25813765,27488283,24560402,24949710,27241509,26110269,24662662,25502235,24056689,25375664,26016363,24610631,26308684,24656288,27396777,27390285,25537917,24525046,24906252,25499341,25703961,24852295,26604214,25488133,27636922,26210752,24122797,27492539,25164220,27890471,25324251,27329400,27492657,24304538,24813561,23956211,24431095,27189212,27141088,26386921,25028167,26242227,24177203,25909652,24817695,24113042,25857307,25657022,26169331,24769543,26911132,24342407,27784026,24373913,26911363,26066077,27745818,24646647,27786148,27486164,24565387,27067479,24861134,26699168,27063071,26250495,24290869,24366914,26152132,25812438,25413133,24637559,25367243,23990195,23984728,23912836,25823269,24062200,25710636,24338088,27095529,26381054

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=27082148,27321790,25169631,24439982,24717626,24252943,25457902,23927483,25277144,26511460,25548342,24342198,27788389,27581344,26113687,24075485,24439983,24311231,24930531,25820525,24815795,24613044,25342738,24094431,25008885,26888150,27835048,26143540,27887774,26905703,26414380,25614164,26195220,30640987,27720494,25286913,25178687,24525158,26324844,24225307,27136008,27021174,24931066,20037100,21956607,20662820,22333215,22333128,19720810,21418744,22203134,21467847,20530034,21828950,23132395,23403476,20583396,22825900,20430262,21626431,23883378,20585011,22985755,21757816,23498778,20418368,22936711,20716845,22362846,20133511,19866403,22898992,20359596,19723905,20669672,20171140,21546436,19686085,22719060,21733953,22759797,23850902,20418789,19609055,21077779,23100508,20113907,22458347,23900878,21812736,21097329,21873802,19909879,23281754,20697180,21781512,20056597,22886553,20405117,22385756

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=22628490,22833239,22819097,23907471,23511365,21256048,21225278,20066882,21304214,22506623,23328389,20585012,23429588,20854331,23568224,20811346,23382484,22776744,19809026,20637015,22543334,20339976,22795519,21471085,22079088,22818063,20206543,23669822,23188123,20854807,21948962,20398563,20022264,23611572,19786728,20150839,23817130,22932074,22170862,21252018,21802156,23744849,23429918,23470352,22390966,23018765,21785860,20352167,22777623,20810512,21034646,23161146,22055098,20923245,21718165,23846611,21884013,21597070,22127844,21415281,21109757,21426745,21418907,19875684,23720841,21362526,23700881,20537740,19538671,21757825,22157844,19644505,22985202,19755415,22555267,22057829,20558556,21081251,21531577,22362844,21527360,22275389,21198560,23900048,23307827,23759655,23370599,23558739,23745440,22505056,20015974,20412435,23361871,20384387,22128226,21256308,16326679,18977464,19251782,16574495

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=16791109,17400684,16982941,18577826,18022001,18959679,19067803,17045906,15969891,15601804,15791043,16507859,17418297,17540215,16284226,16236930,16209972,16242948,18466198,18797697,19381522,17099026,19126937,18081357,15997632,17010807,18300861,18403670,15823634,15772499,16608951,15725144,17140970,18405704,16492990,18515672,15890512,17785618,16320330,19464064,18572079,18793807,18519919,19386283,17384460,17400672,16801459,15516532,17294349,16837670,18971891,16817045,16544967,17135680,16236896,18187744,15818105,18936500,15671601,18339776,17530638,18854878,16401475,16455814,16982943,18812447,17846056,16275518,16963684,19293199,15838368,16807265,16377296,18305126,15838366,17985403,19335742,17613405,19037604,16291503,16291984,16100159,15738296,18375321,16388195,18094217,18058160,19808511,16946127,17019662,18506008,16151039,15680021,16053970,19080476,17135682,17258153,19170724,19071225,16553285

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=19455620,15738301,16645371,18698881,18448495,16889214,18339792,18657627,16919005,18212696,16264047,15464333,18004709,16880101,17095973,16615363,19470885,16563327,19096060,18040672,17439935,16211399,17965512,17223329,19489942,16869470,16078956,18022071,16142516,16778286,16796928,19237088,17121051,16978159,18503929,17012628,15811199,16537868,15747271,17261956,17268158,15750042,18582809,18577825,16253609,16697324,17505045,17175419,16422873,17151415,17006704,17426372,17234519,16917115,15879483,15890561,17804899,17288693,17097500,16875154,18687494,17165160,17030688,17174418,18812445,18625676,18388356,16100158,15956137,16899485,17368113,15731541,16553284,9543369,11550110,11591616,11551870,9366158,14707556,11061392,11789281,9535437,11356783,12563046,11482708,14630619,10892945,11564390,14963694,10830921,15176691,9366762,9277638,9036320,9698961,12654608,14568893,14617591,9768937,14607201

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=12374874,9118516,9465314,9255192,15018019,9485226,15052203,8952581,8960484,9028089,12151469,9890198,11076816,12006444,11843735,9187994,12227646,14651539,11367651,8864793,15289213,12270852,12021227,10719942,11981147,11985913,11981107,8992036,11436677,15214555,12043317,9661544,10520811,15461067,9407564,14555558,11877350,9463708,12296412,8905421,9209604,11208407,11435053,15337703,10355071,15243302,10967899,12760589,9436499,11713133,12354470,15233837,9876980,9825232,9012898,12849677,9716143,10548198,10093074,14618096,14682408,12949023,10368453,11353547,11094038,15055997,11883385,9926800,8765628,12047097,10636277,12900346,9817171,11726420,9347001,9361342,11878269,14718403,14530859,9032045,11434085,11176154,9297420,14520850,12775969,9172849,9469999,10429000,14571638,8759092,11426360,11312483,10821815,8933403,9535073,11149275,11307761,9258481,12838471,10585301

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=9832490,12065350,10720597,9297427,10676695,15358690,11093630,10674757,15364856,9573500,8621193,10861321,9372661,9109008,10545552,14697102,15302727,9327314,9480993,12709727,10527195,11282761,12149694,10869264,15087298,11897647,9762799,11907289,8633936,12358324,12630825,14571608,11926786,12126827,8872846,12658208,10391962,14617263,8990884,12821234,12706935,10430735,11823657,11197402,10908213,9444024,9689394,9890849,9773137,10074645,15357561,10386400,9314423,9187185,11304651,9591925,10357326,12673257,10783653,8895267,9065248,1612153,2511680,7944074,35062,8647585,8727430,2029011,8301459,735294,2881481,7491647,3068741,8222534,8869672,2692332,6148036,1944470,3069486,1536439,3081020,1307892,1677820,1395760,8705151,1873513,3802928,3925746,8592525,1713987,8475936,7646090,7915703,1426824,7004742,6120142,3087241,3289874,7737631,7201271

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=28228114,28167746,27457511,28176907,28069009,28090536,27932335,27884398,27418685,27886187,27875533,27857151,27192969,27435894,27812547,27770771,27703345,27220694,27482885,27462149,27301469,22360383,27117455,27072860,7020727,6362688,8658378,15259082,26501464,26231884,26510696,26135803,26493786,24444659,26590128,24296902,26553699,26469842,26291092,26514759,26310984,25727860,26177174,26084934,25581063,24566799,25850750,25615959,25496504,25641509,25793960,25421879,25644810,25122150,25478953,24991733,24901386,25326008,25176512,22858181,25180165,24987174,24916204,19808294,24498123,23751938,24410934,23985411,24217302,23478192,23443723,23795737,23799908,23653230,23547564,23531633,23496856,22970909,22936191,22963001,22748515,22917554,22447809,22709477,21593252,22676304,20010555,22354598,22189899,21393391


https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=3665737,2879938,2481883,2671486,2013178,7651780,2686485,7955262,3874774,2148281,2286205,7805238,7591009,8397997,2679035,6313787,3530649,3292811,2906956,6107056,2529059,598301,2878953,2334625,7842196,2965386,9181889,3076136,7887106,2209028,8604260,1483797,8087937,7515999,7841754,8545223,2764287,1489073,8620717,2688602,6791673,6342924,1533165,8024129,8518380,2958810,7572624,8645132,3137066,2147842,7004797,3813189,1742818,8045844,3531038,8650414,7841724,7791294,8547558,2736920,2896506,768906,8004302,6991196,812443,1572442,8572790,3917660,8672312,6804108,1745853,3909840,8061266,3003729,2011406,3072444,8205828,3511930,1269432,6398782,2389073,2874735

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=1288902,7718754,7735594,4910309,8357359,8607668,4026044,7548800,349928,2037705,8530849,282070,6375329,7658874,7930234,2523661,4050842,2511776,9007799,8059778,8301443,1451645,7581061,8847877,2933999,7600869,7736741,7991904,3143277,3671896,8100152,17171950,8178023,8159035,2147835,371856,8420736,7602819,6220895,8129541,2131864,2102131,6354031,2759546,7721974

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=11353932,21527519,15050507,21600062,21049284,21219411,21146637,21249218,20581169,20558628,20227301,20175775,20460548,20130830,19940079,20142023,19875683,20042085,20078609,20002090,19536844,18985812,19337745,19375522,19552747,17149459,19335916,18955380,18955380,19023114,19183804,18783596,18055477,7796235,18294393,17472992,17707133,16042696,16423665,16980516,16434427,16793845,15761050,9059470,16061711,12832683,12668787,10639526,11711462,11083748,10722538,9538312,6994766,3326634,8703666,16129028,16092961,15850486,1467051,7495344

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids=11737925,34167677,34627582,34624174,34624174,33735594,33878341,34497295,34373566,33934536,34302745,34262260,34199176,31513270,33625502,33642835,33536021,33415555,33167773,33078386,33293566,30761523,32659755,32912157,32639468,32904701,32857988,32755471,29169546,32680480,32586396,32586396,32161055,32299845,32147955,31765604,30633959,32034015,31843232,30418047,31790543,31343124,31655622,31222765,31511080,29108421,21678455,25068163,31280830,29903860,30632656,30923187,30982349,30636484,30554141,30958848,30624371,30628073,30586694,30567595,30103548,30052230,29750475,21940766,29625733,29763468,24365774,29498925,29608587,29590122,29025899,29707567,29510792,28661697,28963118,28739973,29307087,29282032,25998068,29200842,27757709,28787715,28818881,28874133,25367310,28701597,28039187,28100544,28011757,28556581,28683960,28213957,28676038,28236024,28329315,26756977,27783213,28286762,27602993

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmcids=PMC3198486,PMC5991545,PMC3694514,PMC3560654,PMC3628587,PMC3697070,PMC6652345,PMC2063570,PMC1747498,PMC1208953,PMC1112616,PMC1860020,PMC1769173,PMC1215497,PMC1934985,PMC1861451,PMC2111639,PMC1798307,PMC2683328,PMC2071948,PMC2564804,PMC2292676,PMC2732568,PMC2592739,PMC2655431,PMC2655431,PMC2761583,PMC2635077,PMC2671488,PMC2728149,PMC2629563,PMC1758488,PMC1728597,PMC1745679,PMC1729371,PMC31579,PMC1729511,PMC1730044,PMC96124,PMC1746623,PMC1746721,PMC4724967,PMC3138203,PMC1025687,PMC1386207,PMC1430134,PMC482584,PMC481475,PMC2528455,PMC1009988,PMC472808,PMC1365215

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmcids=PMC4481574,PMC4143816,PMC4662848,PMC4257593,PMC4414424,PMC4500327,PMC4067429,PMC4071704,PMC4211421,PMC5551563,PMC5706646,PMC4335132,PMC4532822,PMC4418312,PMC4411192,PMC4322799,PMC6649535,PMC4285077,PMC4168058,PMC4664609,PMC4368561,PMC4648183,PMC4654874,PMC4452035,PMC4504224,PMC4680166,PMC4692286,PMC4642205,PMC5013094,PMC4604065,PMC4541971,PMC5284330,PMC4940019,PMC4689180,PMC4629509,PMC5351320,PMC4831074,PMC4847358,PMC5094534,PMC4911900,PMC5114641,PMC5122864,PMC5168661,PMC4642211,PMC4722842,PMC5328182,PMC5038574,PMC5698371,PMC5075402,PMC5331109,PMC5085611,PMC5119730,PMC5167631,PMC5142256,PMC5088231,PMC5004960,PMC5529957,PMC5530350,PMC3283528,PMC3126789,PMC3420250,PMC3528474,PMC2807428,PMC3446533,PMC3610124,PMC2805874,PMC2805863,PMC2686069,PMC3040551,PMC2766189,PMC2819977,PMC3291104,PMC4001820,PMC2914575,PMC3034822,PMC3508005,PMC3485404,PMC4167616,PMC3427737,PMC3508003,PMC4904720,PMC3752591,PMC3906683,PMC2850529,PMC3282010,PMC3680653,PMC3040815,PMC3561685,PMC3653687,PMC3694004,PMC3018414,PMC2954311,PMC3148167,PMC2817812,PMC3001255,PMC2874452,PMC3001259,PMC2822592

https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmcids=PMC5223547,PMC5529961,PMC5293990,PMC5261599,PMC5434971,PMC5400052,PMC5496590,PMC5520668,PMC5535573,PMC5593379,PMC5586020,PMC5800854,PMC5867365,PMC5702173,PMC5772136,PMC5929218,PMC5745920,PMC5873992,PMC5880383,PMC5970955,PMC7397551,PMC5953495,PMC6714630,PMC6613216,PMC5637286,PMC5783786,PMC5803650,PMC6913087,PMC5529739,PMC6467240,PMC6299931,PMC6314514,PMC6395424,PMC6607487,PMC5329652,PMC6497345,PMC5322647,PMC5510076,PMC6551213,PMC6453465,PMC6530970,PMC6620412,PMC6853153,PMC5206408,PMC7963136,PMC6815440,PMC7068822,PMC6902175,PMC6986988,PMC5837832,PMC7045190,PMC7160497,PMC7213337,PMC5863312,PMC6026063,PMC7483887,PMC7592949,PMC6078018,PMC7429358,PMC7457596,PMC7488149,PMC6121445,PMC7756901,PMC7856757,PMC8456720,PMC6452473,PMC8449004,PMC8318446,PMC9057455,PMC7047892,PMC8273902,PMC8352969,PMC8426395,PMC8500343,PMC8500343,PMC8514406,PMC7651846,PMC6783595,PMC6739949,PMC7213318,PMC7316574,PMC7316574,PMC7367256,PMC7447256,PMC7763730,PMC7722929,PMC7790729,PMC7905693,PMC7901050,PMC8231632,PMC8294807,PMC3829489,PMC3807672,PMC4654119,PMC3909124,PMC3902062,PMC5947992,PMC4219341,PMC4219342